In [52]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
from dotenv import load_dotenv
import pickle
import pandas as pd

load_dotenv("./.env")

client_id = os.environ.get("SPOTIFY_CLIENT_ID")
client_secret = os.environ.get("SPOTIFY_CLIENT_SECRET")

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id, client_secret), requests_timeout=45)

In [61]:
def load(filename = "filename.pickle"): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 

    except FileNotFoundError: 
        print("File not found!")

kmeans = load("kmeans4000.pickle")
scaler = load("scaler.pickle")

def getClusterIds(_df):
    # Predict clusters using KMeans
    cluster_ids = kmeans.predict(_df)
    return cluster_ids

In [72]:
# Final UI:
# - Input: song name
inpt = input("Enter song name: ")
# - Search for song in Spotify
results = sp.search(q=inpt, limit=1)
# - Get song id
song_id = results["tracks"]["items"][0]["id"]
# - Get song features
inpt_features = sp.audio_features(song_id)
df = pd.DataFrame(inpt_features)[["danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]]
X_normalized = scaler.transform(df)


# - Get cluster id
inpt_cluster = getClusterIds(X_normalized)


In [73]:
# - Get songs in cluster
print(inpt_cluster)
df2 = pd.read_csv("features_clustered.csv")
songs_in_cluster = df2[df2["cluster_id"] == inpt_cluster[0]]["id"].tolist()
song_names = sp.tracks(songs_in_cluster[0:10])["tracks"]
song_names_legible = [i["name"] + ' - ' + i["artists"][0]["name"] for i in song_names]
song_names_legible

[3371]


['Nesesari - Kizz Daniel',
 'Brega do Amor - Aline Silva',
 'Bate Lata - Banda Beijo',
 "3'aza - Omar Alabdallat",
 'Lenguas - Bobby Cruz',
 'Ganges gawa de, koi wo shita. - Nanahira',
 'This Old Man - The Wiggles',
 'Paisa (From "De Dana Dan") - Manak-E',
 'Yur Luv - Tekno',
 'Daddy Yo - Wizkid']

In [74]:
# - Display input song
from IPython.display import IFrame
IFrame(src='https://open.spotify.com/embed/track/' + song_id, width=300, height=380)

In [75]:
# - Display first song in cluster
IFrame(src='https://open.spotify.com/embed/track/' + songs_in_cluster[0], width=300, height=380)